In [13]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from src import const
from pathlib import Path
df_params = pd.read_excel('data/demo(1).xlsx')
df_params0 = pd.read_excel('data/demo.xlsx')

In [16]:
df_params.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   型号            16 non-null     object 
 1   采购成本          16 non-null     int64  
 2   运费及税金￥        16 non-null     float64
 3   最新亚马逊配送费（美金）  16 non-null     float64
 4   VAT20%
（本地国)  16 non-null     int64  
 5   折损（退货率）       16 non-null     float64
 6   促销折扣费用        16 non-null     int64  
 7   PPC推广费用       16 non-null     int64  
 8   汇率            16 non-null     float64
 9   价格下界          16 non-null     float64
 10  价格上界          16 non-null     float64
 11  价格下界时销量       16 non-null     float64
 12  价格上界时销量       16 non-null     int64  
dtypes: float64(7), int64(5), object(1)
memory usage: 1.8+ KB


In [43]:
df_params0

,型号,采购成本,运费及税金￥,最新亚马逊配送费（美金）,VAT20%\n（本地国),折损（退货率）,促销折扣费用,PPC推广费用,汇率,价格下界,价格上界,价格下界时销量,价格上界时销量
0,0BLK,65,10.00,7.09,0,0.1049,0,2,7.1,34,35,127,60
1,0BLU,65,10.00,7.09,0,0.1213,0,0,7.1,34,40,24,20
2,0PUR,67,10.00,7.09,0,0.1218,0,0,7.1,34,45,7,4
3,0BBU,65,10.00,7.09,0,0.0940,0,0,7.1,34,41,4,2
4,0PNK,63,10.00,7.17,0,0.0817,0,0,7.1,34,42,4,2
5,0WHT,63,10.00,7.17,0,0.1239,0,0,7.1,34,43,2,1
6,0PBU,69,10.00,7.17,0,0.1161,0,0,7.1,34,44,7,4
7,0AGN,69,10.00,7.17,0,0.0815,0,0,7.1,34,45,11,1
8,0LPR,69,10.00,7.17,0,0.0765,0,0,7.1,34,46,4,2
9,2BLK,65,10.00,6.77,0,0.1513,0,0,7.1,34,47,18,10


In [5]:

n_product = len(df_params)

df_params['slope'] = (df_params['价格上界时销量'] - df_params['价格下界时销量']) / (df_params['价格上界'] - df_params['价格下界'])
df_params['intercept'] = df_params['价格下界时销量'] - df_params['slope'] * df_params['价格下界']

# 创建模型
model = pyo.ConcreteModel()


def price_bounds_rule(model, i):
    return df_params.loc[i, '价格下界'], df_params.loc[i, '价格上界']

# 定义变量
model.prices = pyo.Var(range(n_product), domain=pyo.NonNegativeReals, bounds=price_bounds_rule)


In [8]:
# 定义目标函数
def total_profit_rule(model):
    a = df_params['slope']
    b = df_params['intercept']

    # FBA佣金（本地国)
    fba_commission = [model.prices[i] * 0.15 for i in range(n_product)]
    # 人工6%
    manual_fee = [model.prices[i] * 0.06 for i in range(n_product)]
    # 仓储成本2%
    storage_fee = [model.prices[i] * 0.02 for i in range(n_product)]
    # 实得收入
    product_income = [model.prices[i] * (1 - df_params['折损（退货率）'][i]) - df_params['最新亚马逊配送费（美金）'][i] -
                        fba_commission[i] - df_params['VAT20%\n（本地国)'][i] - df_params['促销折扣费用'][i] -
                        df_params['PPC推广费用'][i] - manual_fee[i] - storage_fee[i] for i in range(n_product)]
    # 毛利润 rmb
    product_profit_rmb = [product_income[i] * df_params['汇率'][i] - df_params['采购成本'][i] -
                            df_params['运费及税金￥'][i] for i in range(n_product)]

    profit = sum(product_profit_rmb[i] * (a[i] * model.prices[i] + b[i]) for i in range(n_product))
    return profit

model.total_profit = pyo.Objective(rule=total_profit_rule, sense=pyo.maximize)

# 求解模型
solver = pyo.SolverFactory('ipopt')
result = solver.solve(model)

# 输出优化结果
optimized_prices = [pyo.value(model.prices[i]) for i in range(n_product)]
max_profit = pyo.value(model.total_profit)

(type=<class 'pyomo.core.base.objective.ScalarObjective'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.objective.ScalarObjective'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [23]:
max_profit

-1059390.7578069053

In [12]:
df_params

,型号,采购成本,运费及税金￥,最新亚马逊配送费（美金）,VAT20%\n（本地国),折损（退货率）,促销折扣费用,PPC推广费用,汇率,价格下界,价格上界,价格下界时销量,价格上界时销量,slope,intercept
0,100100PNK,65,7.47,6.05,0,0.06,0,260,7.1,23.99,24.00,600.0,600,0.00,600.0000
1,100100BLK,65,7.47,6.07,0,0.10,0,0,7.1,26.99,29.99,544.0,340,-68.00,2379.3200
2,100100BLU,65,7.47,6.05,0,0.10,0,0,7.1,29.99,33.99,100.8,63,-9.45,384.2055
3,100100BRW,65,7.47,6.48,0,0.08,0,0,7.1,29.99,33.99,86.4,54,-8.10,329.3190
4,100100GRN,65,7.47,5.92,0,0.08,0,0,7.1,29.99,33.99,56.0,35,-5.25,213.4475
5,100100GRY,65,7.47,6.21,0,0.09,0,0,7.1,29.99,33.99,65.6,41,-6.15,250.0385
6,100100MU1,65,7.47,5.92,0,0.07,0,0,7.1,29.99,33.99,41.6,26,-3.90,158.5610
7,100100PUR,65,7.47,6.48,0,0.07,0,0,7.1,29.99,33.99,16.0,10,-1.50,60.9850
8,100100BEG,65,7.47,6.77,0,0.07,0,0,7.1,26.99,31.99,44.8,28,-3.36,135.4864
9,100100KHA,65,7.47,6.85,0,0.10,0,0,7.1,29.99,33.99,89.6,56,-8.40,341.5160


In [24]:
prices = optimized_prices
#((0.9*prices[2] - 6.05 - 0.15*prices[2] - 0.06*prices[2] - 0.02*prices[2])*7.1 - 65 - 7.47)*(-9.44999999999999*prices[2] + 384.2054999999997)

3019.8915896527374

In [45]:
((0.94*prices[0] - 6.05 - 0.15*prices[0] - 260 - 0.06*prices[0] - 0.02*prices[0])*7.1 - 65 - 7.47)#*(0.0*prices[0] + 600.0)
# ((0.9*prices[1] - 6.07 - 0.15*prices[1] - 0.06*prices[1] - 0.02*prices[1])*7.1 - 65 - 7.47)*(-68.0*prices[1] + 2379.3199999999997) 
# ((0.9*prices[2] - 6.05 - 0.15*prices[2] - 0.06*prices[2] - 0.02*prices[2])*7.1 - 65 - 7.47)*(-9.44999999999999*prices[2] + 384.2054999999997) 
# ((0.92*prices[3] - 6.48 - 0.15*prices[3] - 0.06*prices[3] - 0.02*prices[3])*7.1 - 65 - 7.47)*(-8.099999999999994*prices[3] + 329.31899999999985) 
# ((0.92*prices[4] - 5.92 - 0.15*prices[4] - 0.06*prices[4] - 0.02*prices[4])*7.1 - 65 - 7.47)*(-5.249999999999996*prices[4] + 213.44749999999985)
# ((0.91*prices[5] - 6.21 - 0.15*prices[5] - 0.06*prices[5] - 0.02*prices[5])*7.1 - 65 - 7.47)*(-6.149999999999993*prices[5] + 250.03849999999977) 
# ((0.9299999999999999*prices[6] - 5.92 - 0.15*prices[6] - 0.06*prices[6] - 0.02*prices[6])*7.1 - 65 - 7.47)*(-3.899999999999997*prices[6] + 158.5609999999999) 
# ((0.9299999999999999*prices[7] - 6.48 - 0.15*prices[7] - 0.06*prices[7] - 0.02*prices[7])*7.1 - 65 - 7.47)*(-1.4999999999999987*prices[7] + 60.98499999999996)
# ((0.9299999999999999*prices[8] - 6.77 - 0.15*prices[8] - 0.06*prices[8] - 0.02*prices[8])*7.1 - 65 - 7.47)*(-3.3599999999999994*prices[8] + 135.48639999999997)
# ((0.9*prices[9] - 6.85 - 0.15*prices[9] - 0.06*prices[9] - 0.02*prices[9])*7.1 - 65 - 7.47)*(-8.399999999999991*prices[9] + 341.51599999999974) 
# ((0.9299999999999999*prices[10] - 6.07 - 0.15*prices[10] - 0.06*prices[10] - 0.02*prices[10])*7.1 - 65 - 7.47)*(-6.749999999999994*prices[10] + 274.4324999999998)
# ((0.95*prices[11] - 5.92 - 0.15*prices[11] - 0.06*prices[11] - 0.02*prices[11])*7.1 - 65 - 7.47)*(-4.649999999999996*prices[11] + 189.05349999999987)
# ((0.92*prices[12] - 5.92 - 0.15*prices[12] - 0.06*prices[12] - 0.02*prices[12])*7.1 - 65 - 7.47)*(-5.999999999999995*prices[12] + 243.93999999999983)
# ((0.89*prices[13] - 5.92 - 0.15*prices[13] - 0.06*prices[13] - 0.02*prices[13])*7.1 - 68 - 8.14)*(-10.6*prices[13] + 529.894)
# ((0.88*prices[14] - 6.48 - 0.15*prices[14] - 0.06*prices[14] - 0.02*prices[14])*7.1 - 68 - 8.14)*(-6.0*prices[14] + 299.94)
# ((0.88*prices[15] - 5.75 - 0.15*prices[15] - 0.06*prices[15] - 0.02*prices[15])*7.1 - 68 - 8.14)*(-3.4*prices[15] + 169.966)

-1830.2169986880574

In [11]:
model.pprint()

1 Var Declarations
    prices : Size=16, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 : 23.99 : 24.000000239974906 :  24.0 : False : False : NonNegativeReals
          1 : 26.99 :  29.64204645755891 : 29.99 : False : False : NonNegativeReals
          2 : 29.99 :  32.46045441784062 : 33.99 : False : False : NonNegativeReals
          3 : 29.99 :   32.4203929371186 : 33.99 : False : False : NonNegativeReals
          4 : 29.99 :  32.01459583586675 : 33.99 : False : False : NonNegativeReals
          5 : 29.99 :  32.39968793123763 : 33.99 : False : False : NonNegativeReals
          6 : 29.99 : 31.847649228845512 : 33.99 : False : False : NonNegativeReals
          7 : 29.99 :  32.24764922803868 : 33.99 : False : False : NonNegativeReals
          8 : 26.99 :  31.99000031197998 : 31.99 : False : False : NonNegativeReals
          9 : 29.99 : 33.057469342715436 : 33.99 : False : False : N

In [ ]:


# 定义目标函数
def total_profit_rule(model):
    a = df_params['slope']
    b = df_params['intercept']

    # FBA佣金（本地国)
    fba_commission = [model.prices[i] * 0.15 for i in range(n_product)]
    # 人工6%
    manual_fee = [model.prices[i] * 0.06 for i in range(n_product)]
    # 仓储成本2%
    storage_fee = [model.prices[i] * 0.02 for i in range(n_product)]
    # 实得收入
    product_income = [model.prices[i] * (1 - df_params['折损（退货率）'][i]) - df_params['最新亚马逊配送费（美金）'][i] -
                        fba_commission[i] - df_params['VAT20%\n（本地国)'][i] - df_params['促销折扣费用'][i] -
                        df_params['PPC推广费用'][i] - manual_fee[i] - storage_fee[i] for i in range(n_product)]
    # 毛利润 rmb
    product_profit_rmb = [product_income[i] * df_params['汇率'][i] - df_params['采购成本'][i] -
                            df_params['运费及税金￥'][i] for i in range(n_product)]

    profit = sum(product_profit_rmb[i] * (a[i] * model.prices[i] + b[i]) for i in range(n_product))
    return profit

model.total_profit = pyo.Objective(rule=total_profit_rule, sense=pyo.maximize)

# 求解模型
solver = pyo.SolverFactory('ipopt')
result = solver.solve(model)

# 输出优化结果
optimized_prices = [pyo.value(model.prices[i]) for i in range(n_product)]
max_profit = pyo.value(model.total_profit)

# 显示优化结果
st.header("步骤 3: 优化结果")
st.subheader("优化后的价格")
opt_price = np.array(optimized_prices).round(2)
df_result = df_params.copy()
df_result['优化价格'] = opt_price
df_result['预期销量'] = (df_result['slope']  * df_result['优化价格'] + df_result['intercept']).round(0).astype(int)
df_result['FBA佣金（本地国)'] = df_result['优化价格']*0.15
df_result['人工6%'] = df_result['优化价格']*0.06
df_result['仓储成本2%'] = df_result['优化价格']*0.02
df_result['得到人民币'] = (df_result['优化价格'] * (1 - df_result['折损（退货率）']) - df_result['最新亚马逊配送费（美金）'] -  df_result['FBA佣金（本地国)'] - df_result['VAT20%\n（本地国)'] - df_result['促销折扣费用'] - df_result['PPC推广费用'] - df_result['人工6%'] - df_result['仓储成本2%']).round(2)

df_result['毛利润 rmb'] = (df_result['得到人民币'] * df_result['汇率'] - df_result['采购成本'] - df_result['运费及税金￥']).round(2)
df_result = df_result.drop(columns=['slope','intercept'])